In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import copy, math

In [3]:
# Load data from .data file 
data = pd.read_csv('./cervical+cancer+behavior+risk/sobar-72.csv')

In [4]:
#  Calculating the percentage of correctly classified examples
# 
def PercentCorrect(Inputs, targets, weights):
    N = len(targets)
    nCorrect = 0
    for n in range(N):
        OneInput = Inputs[n,:]
        if (targets[n]*np.dot(OneInput, weights) > 0):
            nCorrect += 1 
    return 100*nCorrect/N 

In [21]:
# Prepare data set as we want and devide to train and test sets

targets = []
targets = data.iloc[:, 19].values
print(targets.shape)

for i in range(len(targets)):
    if(targets[i] == 0):
        targets[i] = -1

X = data.iloc[:, 0:18+1]

NumDataPerClass = 36

rIndex = np.random.permutation(X.shape[0])  # Shuffle row indices
Xr = X.iloc[rIndex]  # Shuffle rows of X
yr = targets[rIndex]  # Corresponding labels after shuffling

X_train = Xr[0:NumDataPerClass]
y_train = yr[0:NumDataPerClass]
X_test = Xr[NumDataPerClass:2*NumDataPerClass]
y_test = yr[NumDataPerClass:2*NumDataPerClass]
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Ntrain = NumDataPerClass
Ntest = NumDataPerClass

X_train = X_train.values
X_test = X_test.values


from sklearn.preprocessing import StandardScaler

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to your training data (learn mean and standard deviation)
X_train_scaled = scaler.fit_transform(X_train)
# print(X_train_scaled)

# Apply the scaling to your test data (using the parameters learned from training data)
X_test_scaled = scaler.transform(X_test)
# print(X_test_scaled)


(72,)
(36, 19) (36,) (36, 19) (36,)


In [22]:
# Random initialization of weights
w = np.random.randn(19)

# What is the performance with the initial random weights?
print('Initial Percentage Correct: %6.2f' %(PercentCorrect(X_train, y_train, w)))


Initial Percentage Correct:  22.22


In [23]:
def sigmoid(z):

    g = 1/(1+np.exp(-z))
   
    return g

In [24]:
def compute_gradient_logistic(X, y, w, b): 
    
    m,n = X.shape
    dj_dw = np.zeros((n,))                           #(n,)
    dj_db = 0.
    
    for i in range(m):
        z = np.dot(X[i],w) + b
        f_wb_i = sigmoid(z)
        err_i = f_wb_i - y[i]
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i,j] 
        dj_db = dj_db + err_i
    dj_dw = dj_dw/m
    dj_db = dj_db/m
    
    return dj_db, dj_dw

In [25]:
def compute_cost_logistic(X, y, w, b):
    m = X.shape[0]
    cost = 0.0
    for i in range(m):
        z_i = np.dot(X[i],w) + b
        f_wb_i = sigmoid(z_i)
        cost +=  -y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)         
    cost = cost / m
    return cost

In [26]:
def gradient_descent(X, y, w_in, b_in, alpha, num_iters):
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in 
    
    for i in range(num_iters):
        
        dj_db, dj_dw = compute_gradient_logistic(X, y, w, b)
        
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
        
        # Save cost J at each iteration
        J_history.append( compute_cost_logistic(X, y, w, b) )
        
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]}   ")
    
    return w, b, J_history  


In [27]:
w_tmp  = np.zeros_like(X_train[0])
b_tmp  = 0.
alph = 0.1
iters = 10000

w_out, b_out, _ = gradient_descent(X_train, y_train, w_tmp, b_tmp, alph, iters) 
print(f"\nupdated parameters: w:{w_out}, b:{b_out}")

Iteration    0: Cost -96.60011574074075   


C:\Users\User\AppData\Local\Temp\ipykernel_22124\2043581088.py:3: RuntimeWarning: overflow encountered in exp
  g = 1/(1+np.exp(-z))
C:\Users\User\AppData\Local\Temp\ipykernel_22124\1125150510.py:7: RuntimeWarning: divide by zero encountered in log
  cost +=  -y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)
C:\Users\User\AppData\Local\Temp\ipykernel_22124\1125150510.py:7: RuntimeWarning: invalid value encountered in scalar add
  cost +=  -y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)


Iteration 1000: Cost nan   
Iteration 2000: Cost nan   
Iteration 3000: Cost nan   
Iteration 4000: Cost nan   
Iteration 5000: Cost nan   
Iteration 6000: Cost nan   
Iteration 7000: Cost nan   
Iteration 8000: Cost nan   
Iteration 9000: Cost nan   

updated parameters: w:[-3917.15416667 -4945.08888889 -5583.89305556 -3472.62083333
 -5750.67083333 -2639.24861111 -3361.54305556 -1333.48888889
 -4805.97083333 -4889.30833333 -3722.48333333 -6278.41388889
 -5528.30138889 -5750.44027778 -4028.09583333 -6194.99861111
 -7167.22777778 -6889.38333333 -7000.525     ], b:-388.9388888889118
